In [1]:
import os
from tqdm import tqdm
import pandas as pd
from openai import OpenAI, AzureOpenAI
import openai
import constants
import csv
import numpy as np
import concurrent
import time
import json

## Set Up

### Functions

In [2]:
# function for running GPT
def extract_ade_terms(api_source, api_endpoint, gpt_model, system_content, prompt, text, openai_api):
  if api_source == 'OpenAI':
    client = OpenAI(api_key=openai_api)
  elif api_source == 'Azure':
    client = AzureOpenAI(api_key=openai_api, api_version="2023-12-01-preview", azure_endpoint=api_endpoint)
  else:
    raise Exception(f"Unexpected API source requested: {api_source}")
  
  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": system_content},
          {
              "role": "user",
              "content": prompt.format(text)
          }
      ],
      model=gpt_model
  )
  term = chat_completion.choices[0].message.content
  return term

In [3]:
def evaluation_subtype(manual_ades, gpt_output, drug, subtype = 'all'):
    '''
    For a given drug, evaluate the performance of GPT on a given subtype of ADEs. 
    '''
    
    drug_df = manual_ades.query("(drug_name == '{}') & (section_name == 'adverse reactions')".format(drug))
    if subtype == 'exact-meddra': drug_df = drug_df[drug_df.meddra_exact_term == 1]
    if subtype == 'non-meddra': drug_df = drug_df[drug_df.meddra_exact_term == 0]
    if subtype == 'negated': drug_df = drug_df[drug_df.negated_term == 1]
    if subtype == 'discontinuous': drug_df = drug_df[drug_df.discontinuous_term == 1]

    manual = set(drug_df['reaction_string'].to_list())
    gpt_drug = (gpt_output[
        (gpt_output['drug_name'] == drug)
        &
        (gpt_output['section_name'] == "adverse reactions")
        ]["gpt_output"].astype(str)
        .str.lower()
        .str.replace('\n-', ', ')
        .str.split(",").tolist())

    try:
        gpt_drug = [x.strip() for x in gpt_drug[0]]
        gpt_drug = set(gpt_drug)
    except:
        return [drug, subtype, len(manual), len(gpt_drug), np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    
    #overall
    TP = len(manual.intersection(gpt_drug))
    FP = len(gpt_drug.difference(manual))
    FN = len(manual.difference(gpt_drug))
    if TP == 0 and FP == 0:
        precision = np.NAN
    else:
        precision = TP/(TP+FP)
    if TP == 0 and FN == 0:
        recall = np.NAN
    else:
        recall = TP/(TP+FN)
    if precision != 0 and recall != 0:
        f1 = (2 * precision * recall)/(precision + recall)# 2*TP/(2*TP+FP+FN)
    else:
        f1 = np.NAN
    
    if subtype != 'all':
        # these can't be computed for the subtypes
        precision = np.nan
        f1 = np.nan
        FP = np.nan
    
    return [drug, subtype, len(manual), len(gpt_drug), TP, FP, FN, precision, recall, f1]

In [4]:
def evaluation(manual_ades, gpt_output, limit = 1000):
    drugs = gpt_output['drug_name'].unique()
    drugs_set = set()
    results = []
    for drug in tqdm(drugs):

        results.append(evaluation_subtype(manual_ades, gpt_output, drug))
        
    results = pd.DataFrame(results, columns=['drug_name', 'exclude', 'n_manual', 'n_gpt', 'tp', 'fp', 'fn', 'precision', 'recall', 'f1'])
    return results

In [5]:
def evaluation_granular(manual_ades, gpt_output, limit = 1000):
    drugs = gpt_output['drug_name'].unique()
    drugs_set = set()
    results = []
    for drug in tqdm(drugs):
        drugs_set.add(drug)
        if len(drugs_set) > limit:
            break
        
        results.append(evaluation_subtype(manual_ades, gpt_output, drug, subtype = 'all'))
        results.append(evaluation_subtype(manual_ades, gpt_output, drug, subtype = 'exact-meddra'))
        results.append(evaluation_subtype(manual_ades, gpt_output, drug, subtype = 'non-meddra'))
        results.append(evaluation_subtype(manual_ades, gpt_output, drug, subtype = 'negated'))
        results.append(evaluation_subtype(manual_ades, gpt_output, drug, subtype = 'discontinuous'))

    results = pd.DataFrame(results, columns=['drug_name', 'ade_type', 'n_manual', 'n_gpt', 'tp', 'fp', 'fn', 'precision', 'recall', 'f1'])
    return results

### Variables

In [7]:
drug_file = 'data/train_drug_label_text.csv'
manual_file = 'data/train_drug_label_text_manual_ades.csv'
my_max = 10000

In [8]:
drugs = pd.read_csv(drug_file)
manual_ades = pd.read_csv(manual_file)
set_type = drug_file.split('/')[1].split('_')[0] # assuming file follows format "train_..." or "test...."

## Run GPT

In [9]:
outputs = {}

In [18]:
config = json.load(open('./config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint'] 

gpt_model = config[api_source]["gpt_model"]
gpt_model = "gpt-4-0613"


In [19]:
nruns = 1

system_options = {
    "pharmexpert-v0": "You are an expert in pharmacology.",
    "pharmexpert-v1": "You are an expert in medical natural language processing, adverse drug reactions, pharmacology, and clinical trials."
}

prompt_options = {
    "fatal-prompt-v2": """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
The text is :'{}' 
"""
}

system_name = "pharmexpert-v0"
system_content = system_options[system_name]

prompt_name = "fatal-prompt-v2"
prompt = prompt_options[prompt_name]

output_file_basename = '{}_{}_{}_{}_{}'.format(api_source, gpt_model, prompt_name, system_name, set_type)
output_file_basename

'OpenAI_gpt-4-0613_fatal-prompt-v2_pharmexpert-v0_train'

In [20]:
# if there is a max
new_rows = list()
unique_drugs = set()
for i, row in drugs.iterrows():
    unique_drugs.add(row["drug_name"])
    if len(unique_drugs) > my_max: 
        break
    if row['section_name'] != 'adverse reactions':
        continue

    new_rows.append(row)

In [21]:
# run GPT
for i in range(nruns):
    if f"gpt{i}" in outputs:
        print(f"Run {i} already completed and stored. Skipping.")
        continue
    
    if os.path.exists('results/{}_run{}.csv'.format(output_file_basename, i)):
        gpt_output = pd.read_csv('results/{}_run{}.csv'.format(output_file_basename, i))
        outputs[f"gpt{i}"] = gpt_output
        print(f"Run {i} already completed and loaded from disk.")
        continue
    
    start = time.time()
    results = list()
    for row in tqdm(new_rows):
        name, section = row['drug_name'], row['section_name']
        text = row['section_text']
        try:
            gpt_out = extract_ade_terms(api_source, api_endpoint, gpt_model, system_content, prompt, text, api_key)
            time.sleep(5)
            results.append([name, section, gpt_out])
        except Exception as err:
            print(f"Encountered an exception for row: {name} {section}. Error message below:")
            print(err)
            continue
            
    gpt_output = pd.DataFrame(
        [r for r in results if r is not None],
        columns=['drug_name', 'section_name', 'gpt_output']
    )
    end = time.time()

    if gpt_output.shape[0] > 0:
        outputs[f"gpt{i}"] = gpt_output
        gpt_output.to_csv('results/{}_run{}.csv'.format(output_file_basename, i))
    
    print(f"Run: {i}, time elapsed: {end-start}s.")

 65%|██████▌   | 66/101 [21:08<05:24,  9.28s/it]  

Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 10144 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


100%|██████████| 101/101 [28:42<00:00, 17.06s/it]

Run: 0, time elapsed: 1722.6388738155365s.


## Exact Match Algorithm

In [23]:
exact_match_run = True

In [24]:
if not exact_match_run:
    # load the meddra terms
    fh = open('meddra_llt_pt_map.txt')
    reader = csv.reader(fh, delimiter='|')
    header = next(reader)

    meddra_llt_terms = set()
    meddra_pt_terms = set()

    for row in reader:
        meddra_llt_terms.add(row[1].lower())
        meddra_pt_terms.add(row[4].lower())

    fh.close()

    meddra_terms = meddra_llt_terms | meddra_pt_terms
    len(meddra_llt_terms), len(meddra_pt_terms), len(meddra_terms)

    results = list()
    for row in tqdm(new_rows):
        name, section = row['drug_name'], row['section_name']
        text = row['section_text'].lower()
        
        found_terms = set()
        for term in meddra_terms:
            if text.find(term) != -1:
                found_terms.add(term)
        
        exact_out = ', '.join(list(found_terms))
        
        results.append([name, section, exact_out])

    exact_output = pd.DataFrame(
        [r for r in results if r is not None],
        columns=['drug_name', 'section_name', 'gpt_output']
    )
    outputs['exact'] = exact_output

## Evaluation

In [25]:
for method, output in outputs.items():
    results_granular = evaluation_granular(manual_ades, output)
    overall_results = results_granular.groupby('ade_type')['tp', 'fp', 'fn'].apply(sum)
    overall_results['micro_precision'] = overall_results['tp']/(overall_results['tp']+overall_results['fp'])
    overall_results['micro_recall'] = overall_results['tp']/(overall_results['tp']+overall_results['fn'])
    overall_results['micro_f1'] = (2 * overall_results['micro_precision'] * overall_results['micro_recall'])/(overall_results['micro_precision'] + overall_results['micro_recall']) # 2*tp_total/(2*tp_total+fp_total+fn_total)
    macro_results = results_granular.groupby('ade_type')['precision', 'recall', 'f1'].apply(np.mean)
    overall_results['macro_precision'] = macro_results['precision']
    overall_results['macro_recall'] = macro_results['recall']
    overall_results['macro_f1'] = macro_results['f1']
    save_filename = 'results/{}_{}_granular.csv'.format(output_file_basename, method)
    overall_results.to_csv(save_filename)
    print(method)
    print(overall_results)

100%|██████████| 101/101 [00:00<00:00, 154.67it/s]
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall_results = results_granular.groupby('ade_type')['tp', 'fp', 'fn'].apply(sum)
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  macro_results = results_granular.groupby('ade_type')['precision', 'recall', 'f1'].apply(np.mean)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **

gpt0
                   tp     fp      fn  micro_precision  micro_recall  micro_f1  \
ade_type                                                                        
all            3864.0  529.0  1473.0         0.879581      0.724002  0.794245   
discontinuous    49.0    0.0   339.0         1.000000      0.126289  0.224256   
exact-meddra   3423.0    0.0   740.0         1.000000      0.822244  0.902452   
negated          14.0    0.0    18.0         1.000000      0.437500  0.608696   
non-meddra      441.0    0.0   734.0         1.000000      0.375319  0.545792   

               macro_precision  macro_recall  macro_f1  
ade_type                                                
all                   0.856748      0.756106  0.802578  
discontinuous              NaN      0.169050       NaN  
exact-meddra               NaN      0.842792       NaN  
negated                    NaN      0.408730       NaN  
non-meddra                 NaN      0.479649       NaN  


100%|██████████| 101/101 [00:00<00:00, 150.80it/s]
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall_results = results_granular.groupby('ade_type')['tp', 'fp', 'fn'].apply(sum)
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  macro_results = results_granular.groupby('ade_type')['precision', 'recall', 'f1'].apply(np.mean)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **

gpt1
                   tp     fp      fn  micro_precision  micro_recall  micro_f1  \
ade_type                                                                        
all            3816.0  489.0  1521.0         0.886411      0.715008  0.791537   
discontinuous    46.0    0.0   342.0         1.000000      0.118557  0.211982   
exact-meddra   3395.0    0.0   768.0         1.000000      0.815518  0.898386   
negated          15.0    0.0    17.0         1.000000      0.468750  0.638298   
non-meddra      421.0    0.0   754.0         1.000000      0.358298  0.527569   

               macro_precision  macro_recall  macro_f1  
ade_type                                                
all                   0.863214      0.749849  0.809641  
discontinuous              NaN      0.146801       NaN  
exact-meddra               NaN      0.842385       NaN  
negated                    NaN      0.464286       NaN  
non-meddra                 NaN      0.478864       NaN  


100%|██████████| 101/101 [00:00<00:00, 156.08it/s]
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall_results = results_granular.groupby('ade_type')['tp', 'fp', 'fn'].apply(sum)
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  macro_results = results_granular.groupby('ade_type')['precision', 'recall', 'f1'].apply(np.mean)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **

...

100%|██████████| 101/101 [00:00<00:00, 156.42it/s]
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall_results = results_granular.groupby('ade_type')['tp', 'fp', 'fn'].apply(sum)
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  macro_results = results_granular.groupby('ade_type')['precision', 'recall', 'f1'].apply(np.mean)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **

gpt13
                   tp     fp      fn  micro_precision  micro_recall  micro_f1  \
ade_type                                                                        
all            3863.0  588.0  1474.0         0.867895      0.723815  0.789334   
discontinuous    52.0    0.0   336.0         1.000000      0.134021  0.236364   
exact-meddra   3424.0    0.0   739.0         1.000000      0.822484  0.902597   
negated          14.0    0.0    18.0         1.000000      0.437500  0.608696   
non-meddra      439.0    0.0   736.0         1.000000      0.373617  0.543990   

               macro_precision  macro_recall  macro_f1  
ade_type                                                
all                   0.844568      0.750524  0.795905  
discontinuous              NaN      0.138168       NaN  
exact-meddra               NaN      0.842457       NaN  
negated                    NaN      0.376984       NaN  
non-meddra                 NaN      0.474090       NaN  


100%|██████████| 101/101 [00:00<00:00, 156.66it/s]

gpt14
                   tp     fp      fn  micro_precision  micro_recall  micro_f1  \
ade_type                                                                        
all            3963.0  594.0  1374.0         0.869651      0.742552  0.801092   
discontinuous    59.0    0.0   329.0         1.000000      0.152062  0.263982   
exact-meddra   3524.0    0.0   639.0         1.000000      0.846505  0.916873   
negated          15.0    0.0    17.0         1.000000      0.468750  0.638298   
non-meddra      439.0    0.0   736.0         1.000000      0.373617  0.543990   

               macro_precision  macro_recall  macro_f1  
ade_type                                                
all                   0.853476      0.762947  0.815717  
discontinuous              NaN      0.157197       NaN  
exact-meddra               NaN      0.856362       NaN  
negated                    NaN      0.464286       NaN  
non-meddra                 NaN      0.466423       NaN  



/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall_results = results_granular.groupby('ade_type')['tp', 'fp', 'fn'].apply(sum)
/var/folders/l5/4jn07y290nncb4wnyhsq33hh0000gt/T/ipykernel_2051/2659214344.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  macro_results = results_granular.groupby('ade_type')['precision', 'recall', 'f1'].apply(np.mean)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Library/Frameworks/Python.framework/Versi